In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg

In [10]:
xrp = pd.read_csv("data/minute.csv").XRP_price

In [12]:
ar_model = AutoReg(xrp, lags=1).fit()
print(ar_model.summary())

                            AutoReg Model Results                             
Dep. Variable:              XRP_price   No. Observations:                76312
Model:                     AutoReg(1)   Log Likelihood              443334.159
Method:               Conditional MLE   S.D. of innovations              0.001
Date:                Tue, 15 Mar 2022   AIC                        -886662.318
Time:                        18:19:07   BIC                        -886634.590
Sample:                             1   HQIC                       -886653.800
                                76312                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const         2.705e-05   2.32e-05      1.166      0.244   -1.84e-05    7.25e-05
XRP_price.L1     1.0000   3.19e-05   3.14e+04      0.000       1.000       1.000
                                    Roots   

/Users/mgaulia/miniforge3/envs/torch/lib/python3.8/site-packages/statsmodels/tsa/ar_model.py:1064: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(lag_repr) ** -1


XRP BTC
AR ARMA ARIMA GARCH
MIN HOUR DAY